In [8]:
import sys
import os
os.chdir('/Users/rwb/github/topsim')
sys.path.insert(0, '/Users/rwb/github/topsim')
sys.path.insert(0, '/Users/rwb/github/shadow/')
sys.path.append('..')
from algorithms.scheduling import FifoAlgorithm
from core.simulation import Simulation
import test_data

In [35]:
import test_data

In [9]:
test_data.machine_config

'test/data/system_config.json'

In [10]:
import simpy
import time
import os
workflow_file = None
event_file = 'sim.trace'
planning_algorithm = 'heft'
env = simpy.Environment()
tmax = 36  # for starters, we will define telescope configuration as simply number of arrays that exist
salgorithm = FifoAlgorithm()
vis = False

simulation = Simulation(env,
                       test_data.telescope_config,
                        tmax,
                        test_data.machine_config,
                        salgorithm,
                        'heft',
                        event_file,
                        vis)

events = []
not_started = True
start = False


In [31]:
print(test_data.machine_config)

test/data/daligue_pipeline_test_sys.json


In [5]:
os.setcwd()

AttributeError: module 'os' has no attribute 'setcwd'

In [11]:
import time
import simpy

"""
Use 'bokeh' python library to visualise the current state of the system
"""
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.plotting import figure, ColumnDataSource
import json
import numpy as np
import pandas as pd
from bokeh.driving import count
from bokeh.layouts import column, gridplot, row
from bokeh.models import ColumnDataSource, Select, Slider
from bokeh.plotting import curdoc, figure
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.models.widgets import Button, DataTable, TableColumn
from bokeh.server.server import Server
from bokeh.themes import Theme
from threading import Thread
from bokeh.colors import color
from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, PreText, Select
from bokeh.plotting import figure
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature
from flask import Flask, render_template
from bokeh.embed import server_document
from tornado.ioloop import IOLoop
from bokeh.transform import transform
import logging
logger = logging.getLogger(__name__)


In [12]:
def make_document(doc):
    # source = ColumnDataSource({'x': [], 'y': [], 'color': []})
    # x = []
    # y = []

    colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
    mapper = LinearColorMapper(palette=colors, low=0, high=1)
    button = Button(label="Start", button_type="success")
    # create three plots

    plot = {
        'time': [env.now],
        'running': [len(simulation.cluster.running_tasks)],
        'finished': [len(simulation.cluster.finished_tasks)]
    }

    plotdata = ColumnDataSource(plot)

    def update_start():
        if not start:
            start = True

    p = figure(plot_width=400, plot_height=400)
    p.line(x='time', y='running', legend_label='Running tasks', alpha=0.2, line_width=3, color='navy', source=plotdata)
    p.line(x='time', y='finished', legend_label='Finished tasks', alpha=0.8, line_width=2, color='orange', source=plotdata)
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Tasks'

    def update_plot():
        if env:
            updata = {
                'time': [env.now],
                'running': [len(simulation.cluster.running_tasks)],
                'finished': [len(simulation.cluster.finished_tasks)]
            }
            plotdata.stream(updata)

    """
    SETUP TABLE 
    """
    columns = [
        TableColumn(field="name", title="Name"),
        TableColumn(field="start", title="Start Time"),
        TableColumn(field="demand", title="No. Antennas"),
        TableColumn(field="duration", title="Observation Length"),
        TableColumn(field="running", title="Running Status")
    ]

    sourcedata = dict(name=[], start=[], duration=[], demand=[], running=[])
    tablesource = ColumnDataSource(data=sourcedata)
    for observation in simulation.telescope.observations:
        sourcedata['name'].append(observation.name)
        sourcedata['start'].append(observation.start)
        sourcedata['duration'].append(observation.duration)
        sourcedata['demand'].append(observation.demand)
        sourcedata['running'].append(observation.status)
    data_table = DataTable(source=tablesource, columns=columns, width=600, height=280)

    def update_table():
        if env:
            for i, observation in enumerate(simulation.telescope.observations):
                dictionary = {"running": [(i, observation.status)]}
                tablesource.patch(dictionary)

    doc.add_periodic_callback(update_plot, 500)
    doc.add_periodic_callback(update_table, 500)
    button.on_click(update_start)

    # fig.xgrid.grid_line_color = None
    # fig.x_range.start = 0
    # fig.x_range.end = 350
    # fig.legend.orientation = "horizontal"
    # fig.legend.location = "top_center"

    doc.title = "TopSim Dashboard"
    doc.add_root(layout([button], [p, data_table]))
    return doc

In [13]:
logger.debug('Opening Bokeh application on http://localhost:8898/')
apps = {'/': Application(FunctionHandler(make_document))}
server = Server(apps, port=5070)

INFO:bokeh.server.server:Starting Bokeh server version 2.0.2 (running on Tornado 6.0.3)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


In [21]:
server.start()

In [22]:
simulation.start(runtime=-1)

INFO:core.telescope:Observation emu scheduled for 0
INFO:core.telescope:Telescope is now using 36 arrays
INFO:core.buffer:Adding observation emu data to buffer at time 11
INFO:core.telescope:Telescope is now using 0 arrays
INFO:core.telescope:Observation dingo scheduled for 11
INFO:core.telescope:Telescope is now using 18 arrays
INFO:core.scheduler:New observation emu scheduled for processing @ Time: 12
INFO:core.scheduler:emu finished processing @ 17
INFO:core.telescope:Observation vat scheduled for 20
INFO:core.telescope:Telescope is now using 36 arrays
INFO:core.buffer:Adding observation dingo data to buffer at time 26
INFO:core.telescope:Telescope is now using 18 arrays
INFO:core.scheduler:New observation dingo scheduled for processing @ Time: 27
INFO:core.scheduler:dingo finished processing @ 32
INFO:core.buffer:Adding observation vat data to buffer at time 51
INFO:core.telescope:Telescope is now using 0 arrays
INFO:core.scheduler:New observation vat scheduled for processing @ Tim